In [17]:
import numpy as np
import pandas as pd 
import re 
import torch 
import torch.nn as nn
import sklearn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize
from torch.utils.data import DataLoader,Dataset

In [75]:
if torch.cuda.is_available:
    device = 'cuda'
else:
    device = 'cpu'
device

'cuda'

# **document similarty score**
*this one of the fundemental area of NLP there are several methods proposed to do this taks*

**top methods**

1. Cosine Similarity
*here we will be converting documents into vectors and apply for this vector conversion we normally use Term Frequency-Inverse Document Frequency and the dot product between two docs represent the similarity*

*this method doesnt take capture the semantic meaning of words for example teacher and professor are consider like different individual*

2. Word Embedding-Based Methods
*in this method we fine-tune/train numerical word embeddings based on their occurance in a sentence*
*for this there are two widely used architecture are 'Continuous Bag-of-Words' and 'skip gram'*

**Continuous Bag-of-Words**
*predicts the middle word based on surrounding context words. The context consists of a few words before and after the current (middle) word.*

**example**
> i am rohith currenty pursuing btech 

> given "i am rohith ---- pursuing btech" the model will predict the currently

> we will be updating the embedding layer weights through backprop

**skip gram model**
*predicts words within a certain range before and after the current word in the same sentence. A worked example of this is given below.*

**example**

> i am rohith currenty pursuing btech 

> given "pursuing" the model will predict the i am rohith  pursuing btech

> we will be updating embedding layer weights through backprop

**skip gram model is able to get better semantic and Syntactic accuracy then cbow model so we will be implementing the skip gram model to get the word embeddings**

*source https://arxiv.org/pdf/1301.3781.pdf (Efficient Estimation of Word Representations in Vector Space)*

In [18]:
with open('/kaggle/input/document-classification/file.txt','r') as f:
    data = f.read()
data = segments = re.split(r'\n[1-9]', data)
data[1]

' champion products ch approves stock split champion products inc said its board of directors approved a two for one stock split of its common shares for shareholders of record as of april the company also said its board voted to recommend to shareholders at the annual meeting april an increase in the authorized capital stock from five mln to mln shares reuter '

In [20]:
idx = 1
word_to_num_dict = {}
for token in tokenized_corpus:
    for word in token:
        if(word not in word_to_num_dict.keys()):
            word_to_num_dict[word]=idx
            idx+=1
vocab_size = len(word_to_num_dict)
vocab_size

19982

In [58]:
num_to_word_dict = {value: key for key, value in word_to_num_dict.items()}

**the dataset contains 19982 unique words** 

In [45]:
x = []
y = []
for token in tokenized_corpus:
    for i in range(2, len(token) - 2):
        center_word_idx = word_to_num_dict[token[i]]
        context_word_indices = [word_to_num_dict[token[j]] for j in range(i-2, i+3) if j != i] 
        x.append(center_word_idx)
        y.append(context_word_indices)

In [77]:
x = torch.tensor(x)
y = torch.tensor(y)
x.shape , y.shape

/tmp/ipykernel_32/2346925435.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)
/tmp/ipykernel_32/2346925435.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y)


(torch.Size([555557]), torch.Size([555557, 4]))

**Dataset and DataLoader**

In [78]:
class customdataset(Dataset):
    def __init__(self,x,y):
        self.inputs = x
        self.outputs = y
    def __len__(self):
        return len(self.input)
    def __getitem__(self,idx):
        return self.input[idx].to(device),self.output[idx].to(device)

In [79]:
dataset = customdataset(x,y)

In [80]:
Loader = DataLoader(dataset,batch_size=128,num_workers=2)

**Skip gram model**

In [146]:
class skip_gram(nn.Module):
    def __init__(self,vocab_size,dimension):
        super(skip_gram,self).__init__()
        self.dimension=dimension
        self.target_embedding = nn.Embedding(vocab_size,dimension)
        self.context_embedding = nn.Embedding(vocab_size,dimension)
    def forward(self,target,context):
        if(len(target.size())==2):
            target=target.squeeze(axis=1)
        target_embedding = self.target_embedding(target)
        print(target_embedding.shape)
        context_embedding = self.context_embedding(context)
        print(context_embedding.shape)
        return torch.einsum('be,bce->bc', target_embedding,context_embedding)

In [147]:
model = skip_gram(19983,5)

In [148]:
import torch.optim as optim 
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [149]:
model.eval()

skip_gram(
  (target_embedding): Embedding(19983, 5)
  (context_embedding): Embedding(19983, 5)
)

In [152]:
test = model(torch.tensor([1,2]).view(2,1),torch.tensor([[1000,3,4,5],[3,6,7,8]]).view(2,4))
test

torch.Size([2, 5])
torch.Size([2, 4, 5])


tensor([[ 1.3086,  3.3790, -0.4160,  3.6633],
        [ 2.8049,  0.1689, -0.5356,  2.8980]], grad_fn=<ViewBackward0>)